### Import libraries

In [2]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import csv
from bs4 import BeautifulSoup

### Define useful functions

In [207]:
def extract_review_body(review_query):
    if review_query != None:
        return review_query.find("span", itemprop="reviewBody").get_text()
    else:
        return None

def extract_review_staydate(review_query):
    if review_query != None:
        return review_query.get_text().strip('\n').strip('Stayed in ')
    else:
        return None
    
def de_emojify(inputString):
    if inputString!=None:
        return inputString.encode('ascii', 'ignore').decode('ascii')
    else:
        return inputString

### Specify user agent

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

### Get names and urls of hotels in NYS

In [5]:
next_page=True
ip=1
hotel_results_offset=0
hotel_info_list = []
while next_page:    
    url= 'http://www.booking.com/reviews/region/new-york-state.html?offset='+str(hotel_results_offset)+'&'
    response = requests.get(url,headers=headers)
    hotel_info = BeautifulSoup(response.text, 'html.parser')
    hotel_info_page = hotel_info.find_all(class_=['rlp-main-hotel','li'])
    hotel_info_list.extend(hotel_info_page)
    ip+=1
    hotel_results_offset+=30
    if hotel_info_page==[]:
        next_page=False

In [36]:
#hotel_url_list = []
#hotel_name_list = []
nys_hotel_info_list_csvname = './datasets/nys_hotel_info_list.csv'

with open(nys_hotel_info_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','hotel_url'])
    
    for hotel_info_item in hotel_info_list:   
        hotel_name = hotel_info_item.find(class_ = 'rlp-main-hotel__hotel-name-link').get_text()
        hotel_url = hotel_info_item.find(class_ = 'rlp-main-hotel__hotel-name-link')['href'].split('.html?')[0]

        #hotel_name_list.append(hotel_name)
        #hotel_url_list.append(hotel_url)
        writer.writerow([hotel_name,hotel_url])

### Get names and urls of hotels in Arizona

In [62]:
next_page=True
ip=1
hotel_results_offset=0
hotel_info_list = []
while next_page:    
    url= 'http://www.booking.com/reviews/region/arizona.html?offset='+str(hotel_results_offset)+'&'
    response = requests.get(url,headers=headers)
    hotel_info = BeautifulSoup(response.text, 'html.parser')
    hotel_info_page = hotel_info.find_all(class_=['rlp-main-hotel','li'])
    hotel_info_list.extend(hotel_info_page)
    ip+=1
    hotel_results_offset+=30
    if hotel_info_page==[]:
        next_page=False

In [63]:
#hotel_url_list = []
#hotel_name_list = []
arizona_hotel_info_list_csvname = './datasets/arizona_hotel_info_list.csv'

with open(arizona_hotel_info_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','hotel_url'])
    
    for hotel_info_item in hotel_info_list:   
        hotel_name = hotel_info_item.find(class_ = 'rlp-main-hotel__hotel-name-link').get_text()
        hotel_url = hotel_info_item.find(class_ = 'rlp-main-hotel__hotel-name-link')['href'].split('.html?')[0]

        #hotel_name_list.append(hotel_name)
        #hotel_url_list.append(hotel_url)
        writer.writerow([hotel_name,hotel_url])

### Get reviews of hotels in NYS

In [233]:
nys_hotel_info_list_csvname = './datasets/nys_hotel_info_list.csv'
df_nys_hotel_info_list = pd.read_csv(nys_hotel_info_list_csvname)

In [234]:
nys_hotel_reviews_list_csvname = './datasets/sample3_nys_hotel_reviews_list.csv'

with open(nys_hotel_reviews_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','review_date','review_item_user_review_count',
                     'review_score_badge','review_item_header_content','review_info_tag','review_staydate',
                     'review_pos','review_neg'])
    
    for idx,hotel_url_item in enumerate(df_nys_hotel_info_list['hotel_url'][:1000]):

        next_page=True
        ip=1
        hotel_reviews_list = []
        while next_page:
            url= 'http://www.booking.com/reviews/us/hotel/'+hotel_url_item.split('/hotel/us/')[1]+'.html?page='+str(ip)+';r_lang=en'
            response = requests.get(url,headers=headers)
            hotel_reviews = BeautifulSoup(response.text, 'html.parser')
            hotel_reviews_page = hotel_reviews.find_all(class_=['review_item','li'])
            hotel_reviews_list.extend(hotel_reviews_page)
            ip+=1
            if hotel_reviews_page==[]:
                next_page=False
            
        for hotel_reviews_item in hotel_reviews_list:   
            review_date = hotel_reviews_item.find(class_ = 'review_item_date').get_text().strip('\n').strip('Reviewed: ')
            #reviewer_name = hotel_reviews_item.find(class_ = 'reviewer_name').find("span", itemprop="name").get_text()
            #reviewer_country = hotel_reviews_item.find(class_ = 'reviewer_country').find("span", itemprop="name").get_text()
            review_item_user_review_count = hotel_reviews_item.find(class_ = 'review_item_user_review_count').get_text().strip('\n')
            review_score_badge = hotel_reviews_item.find(class_ = 'review-score-badge').get_text().strip('\"')
            review_item_header_content = hotel_reviews_item.find(class_ = 'review_item_header_content').get_text().strip('\n').replace('\u201c','').replace('\u201d','')
            review_info_tag = [tag.get_text().strip('\n')[2:] for tag in hotel_reviews_item.find_all(class_=['review_info_tag','li'])]
            review_staydate = extract_review_staydate(hotel_reviews_item.find(class_ = 'review_staydate'))
            review_pos = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'review_pos')))
            review_neg = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'review_neg')))
        
            writer.writerow([df_nys_hotel_info_list['hotel_name'][idx],review_date,
                             review_item_user_review_count,review_score_badge,review_item_header_content,
                             review_info_tag,review_staydate,review_pos,review_neg])

### Get reviews of hotels in Arizona

In [64]:
arizona_hotel_info_list_csvname = './datasets/arizona_hotel_info_list.csv'
df_arizona_hotel_info_list = pd.read_csv(arizona_hotel_info_list_csvname)

In [67]:
arizona_hotel_reviews_list_csvname = './datasets/sample1_arizona_hotel_reviews_list.csv'

with open(arizona_hotel_reviews_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','review_date','review_item_user_review_count',
                     'review_score_badge','review_item_header_content','review_info_tag','review_staydate',
                     'review_pos','review_neg'])
    
    for idx,hotel_url_item in enumerate(df_arizona_hotel_info_list['hotel_url'][300:800]):

        next_page=True
        ip=1
        hotel_reviews_list = []
        while next_page:
            url= 'http://www.booking.com/reviews/us/hotel/'+hotel_url_item.split('/hotel/us/')[1]+'.html?page='+str(ip)+';r_lang=en'
            response = requests.get(url,headers=headers)
            hotel_reviews = BeautifulSoup(response.text, 'html.parser')
            hotel_reviews_page = hotel_reviews.find_all(class_=['review_item','li'])
            hotel_reviews_list.extend(hotel_reviews_page)
            ip+=1
            if hotel_reviews_page==[]:
                next_page=False
            
        for hotel_reviews_item in hotel_reviews_list:   
            review_date = hotel_reviews_item.find(class_ = 'review_item_date').get_text().strip('\n').strip('Reviewed: ')
            #reviewer_name = hotel_reviews_item.find(class_ = 'reviewer_name').find("span", itemprop="name").get_text()
            #reviewer_country = hotel_reviews_item.find(class_ = 'reviewer_country').find("span", itemprop="name").get_text()
            review_item_user_review_count = hotel_reviews_item.find(class_ = 'review_item_user_review_count').get_text().strip('\n')
            review_score_badge = hotel_reviews_item.find(class_ = 'review-score-badge').get_text().strip('\"')
            review_item_header_content = hotel_reviews_item.find(class_ = 'review_item_header_content').get_text().strip('\n').replace('\u201c','').replace('\u201d','')
            review_info_tag = [tag.get_text().strip('\n')[2:] for tag in hotel_reviews_item.find_all(class_=['review_info_tag','li'])]
            review_staydate = extract_review_staydate(hotel_reviews_item.find(class_ = 'review_staydate'))
            review_pos = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'review_pos')))
            review_neg = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'review_neg')))
        
            writer.writerow([df_arizona_hotel_info_list['hotel_name'][idx],review_date,
                             review_item_user_review_count,review_score_badge,review_item_header_content,
                             review_info_tag,review_staydate,review_pos,review_neg])

## EDA - Arizona dataset

In [230]:
df_tmp = pd.read_csv(arizona_hotel_reviews_list_csvname)
df_tmp.tail()

,hotel_name,review_date,reviewer_name,reviewer_country,review_item_user_review_count,review_score_badge,review_item_header_content,review_info_tag,review_staydate,review_pos,review_neg
47590,Quality Inn Holbrook,22 April 2019,Letcherous,Canada,24 reviews,5.4,Beds were the only bright light in our stay,"['Leisure trip', 'Couple', 'Queen Room with Tw...",April 2019,Beds were comfy,Breakfast was mediocre and the tub and toilet ...
47591,Quality Inn Holbrook,16 April 2019,Peg,United States,9 reviews,9.2,"COZY AND PRACTICAL, ESPECIALLY SINCE WE NEED T...","['Leisure trip', 'Couple', 'Queen Room - Non-S...",April 2019,Lots to choose from on the breakfast buffet. W...,They might want to offer tea in with the buffe...
47592,Quality Inn Holbrook,9 April 2019,Stefanie,United States,2 reviews,3.3,Awful,"['Leisure trip', 'Family with young children',...",April 2019,"No food, just coffee is that breakfast?","The owner, bed bugs ate up my baby!!!!!!"
47593,Quality Inn Holbrook,4 April 2019,Graeme,Australia,19 reviews,5.8,back up after another hotel had no vacancies,"['Leisure trip', 'Couple', 'Studio Suite with ...",March 2019,location,price
47594,Quality Inn Holbrook,17 March 2019,Mark,United States,11 reviews,5.0,"Would not stay there again and, perhaps, never...","['Leisure trip', 'Couple', 'Queen Room - Disab...",March 2019,Location,Substandard breakfast. Unable to secure a room...


In [ ]:
#https://scholarship.sha.cornell.edu/cgi/viewcontent.cgi?article=1003&context=chrreports